In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
hostname_to_cpu_arch = {
    "ariel": "Skylake SP",
    "hati": "Sapphire Rapids"
}

In [ ]:
arch_to_experiment = {}

experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('robsize'), experiments))
for hostname, arch in hostname_to_cpu_arch.items():
    hostname_experiments = list(filter(ExperimentFilter.by_host(hostname), experiments))
    hostname_latest = ExperimentFilter.get_latest(hostname_experiments)
    arch_to_experiment[arch] = hostname_latest

arch_experiments = list(arch_to_experiment.values())

arch_to_experiment

In [ ]:
INSTRUCTION_COUNT = 'Number of instructions'

In [ ]:
df = pd.DataFrame()

for arch, experiment in arch_to_experiment.items():
    arch_df = pd.read_csv(experiment.path / 'robsize.csv')
    arch_df["Microarchitecture"] = arch

    df = pd.concat([
        df, arch_df
    ], ignore_index=True)

df["TestType"] = list(map(lambda s: s.split(";")[0], df["TestName"]))
df[INSTRUCTION_COUNT] = df["InstructionCount"]
df

In [ ]:
df_nop = df[df["TestName"] == "nop;single-byte"]
df_nop["MinCyclesNormalised"] = df_nop.groupby("Microarchitecture", group_keys=False, sort=False)["MinCycles"].apply(lambda x: 100*(x-x.min())/(x.max()-x.min()))

plt.rcParams["figure.figsize"] = (7, 5)
ax = sns.lineplot(data=df_nop, x=INSTRUCTION_COUNT, y="MinCyclesNormalised", markers=True, dashes=False, hue="Microarchitecture")
ax.set_xlabel("Number of single byte nop instructions")
ax.set_ylabel("Normalized execution speed of fastest repetition [%]")
ax.set_ylim(0, 100)
ax.set_xlim(200, 520)
ax.set_xticks(range(200, 520, 20))

ax.vlines(x=224, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
ax.text(x=224.5, y=ax.get_ylim()[1]-0.01, rotation=90, s='Skylake SP Reorder Buffer', verticalalignment='top')

ax.vlines(x=512, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
ax.text(x=512.5, y=ax.get_ylim()[1]-0.01, rotation=90, s='Sapphire Rapids Reorder Buffer', verticalalignment='top')

Plotting.savefig(arch_experiments, 'reorder-buffer.pdf', annotations_x_offset=0.05, annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
df_load = df[df["TestType"] == "load"]
df_load["RegisterType"] = list(map(lambda s: s.split(";")[1], df_load["TestName"]))
df_load = df_load[df_load["RegisterType"] == "gp-registers"]
df_load["MinCyclesNormalised"] = df_load.groupby("Microarchitecture", group_keys=False, sort=False)["MinCycles"].apply(lambda x: 100*(x-x.min())/(x.max()-x.min()))
df_load[INSTRUCTION_COUNT] += 2

plt.rcParams["figure.figsize"] = (7, 5)
# ax = sns.lineplot(data=df3, x="InstructionCount", y="MinCyclesNormalised", hue="RegisterType", markers=True, dashes=False, style="Microarchitecture")
ax = sns.lineplot(data=df_load, x=INSTRUCTION_COUNT, y="MinCyclesNormalised", markers=True, dashes=False, hue="Microarchitecture", marker='o')
ax.set_xlabel("Number of load instructions")
ax.set_ylabel("Normalized execution speed of fastest repetition [%]")
ax.set_ylim(0, 50)
ax.set_xlim(62, 250)
ax.set_xticks(range(62, 250, 16))

ax.vlines(x=72, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
ax.text(x=72.5, y=ax.get_ylim()[1]-0.01, rotation=90, s='Skylake SP Load Buffer', verticalalignment='top')

ax.vlines(x=192, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')

ax.vlines(x=240, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
ax.text(x=240.5, y=ax.get_ylim()[1]-0.01, rotation=90, s='Sapphire Rapids Load Buffer', verticalalignment='top')

Plotting.savefig(arch_experiments, 'load-buffer.pdf', annotations_x_offset=0.05, annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
df3 = df[df["TestType"] == "store"]
df3["RegisterType"] = list(map(lambda s: s.split(";")[1], df3["TestName"]))
df3 = df3[df3["RegisterType"] == "gp-registers"]
df3["MinCyclesNormalised"] = df3.groupby("Microarchitecture", group_keys=False, sort=False)["MinCycles"].apply(lambda x: 100*(x-x.min())/(x.max()-x.min()))

plt.rcParams["figure.figsize"] = (7, 5)
# ax = sns.lineplot(data=df3, x="InstructionCount", y="MinCyclesNormalised", hue="RegisterType", markers=True, dashes=False, style="Microarchitecture")
ax = sns.lineplot(data=df3, x=INSTRUCTION_COUNT, y="MinCyclesNormalised", markers=True, dashes=False, hue="Microarchitecture", marker='o')
ax.set_xlabel("Number of store instructions")
ax.set_ylabel("Normalized execution speed of fastest repetition [%]")
ax.set_ylim(0, 50)
ax.set_xlim(50, 118)
ax.set_xticks(range(50, 118, 4))

ax.vlines(x=56, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
ax.text(x=56.5, y=ax.get_ylim()[1]-0.01, rotation=90, s='Skylake SP Store Buffer', verticalalignment='top')

ax.vlines(x=112, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
ax.text(x=112.5, y=ax.get_ylim()[1]-0.01, rotation=90, s='Sapphire Rapids Store Buffer', verticalalignment='top')

Plotting.savefig(arch_experiments, 'store-buffer.pdf', annotations_x_offset=0.05, annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
REGISTER_TYPE = 'Register Type'
NORMALIZED_MIN_CYCLES = 'Normalized execution speed of fastest repetition [%]'

def plot_to_axes(dataframe, ax, test_type: str, alternating: bool):
    df = dataframe[dataframe["TestType"] == test_type]
    if alternating:
        df = df[df["Alternating"] == "alternating"]
    else:
        df = df[df["Alternating"] == "non-alternating"]

    # Filter out zmm register when using mov or cmp
    if test_type in ["mov", "cmp"]:
        df = df[df[REGISTER_TYPE] != "zmm"]

    df[NORMALIZED_MIN_CYCLES] = df.groupby(["Microarchitecture", REGISTER_TYPE], group_keys=False, sort=False)["MinCycles"].apply(lambda x: 100*(x-x.min())/(x.max()-x.min()))
    sns.lineplot(data=df, ax=ax, x=INSTRUCTION_COUNT, y=NORMALIZED_MIN_CYCLES, hue=REGISTER_TYPE, markers=False, dashes=True, style="Microarchitecture")
    ax.set_xlim(100, 400)

    ax.set_ylim(0, 100)

    ax.vlines(x=148, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
    ax.text(x=148.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='Integer (measured)', verticalalignment='bottom')

    ax.vlines(x=132, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
    ax.text(x=132.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='FP (measured)', verticalalignment='bottom')

    ax.vlines(x=180, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
    ax.text(x=180.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='Integer', verticalalignment='bottom')

    ax.vlines(x=168, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey')
    ax.text(x=168.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='FP', verticalalignment='bottom')

    ax.vlines(x=288, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey', linestyles='dashed')
    ax.text(x=288.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='Integer', verticalalignment='bottom')

    ax.vlines(x=240, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey', linestyles='dashed')
    ax.text(x=240.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='Integer (measured)', verticalalignment='bottom')

    # Filter out zmm register when using mov or cmp
    if test_type not in ["mov", "cmp"]:
        ax.vlines(x=220, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey', linestyles='dashed')
        ax.text(x=220.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='FP (512b)', verticalalignment='bottom')

        ax.vlines(x=188, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey', linestyles='dashed')
        ax.text(x=188.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='FP (512b) measured', verticalalignment='bottom')

    ax.vlines(x=320, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey', linestyles='dashed')
    ax.text(x=320.5, y=ax.get_ylim()[0]+0.01, rotation=90, s='FP (256b)', verticalalignment='bottom')

    ax.text(x=274, y=ax.get_ylim()[0]+0.01, rotation=90, s='FP (256b) measured', verticalalignment='bottom')
    ax.vlines(x=282, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], color='grey', linestyles='dashed')

# Filter for the relevant instructions
test_types = ["add", "mov", "cmp", "xor"]
data = df[df["TestType"].isin(test_types)]

data[REGISTER_TYPE] = list(map(lambda s: s.split(";")[1].split('-')[0], data["TestName"]))
data["Alternating"] = list(map(lambda s: s.split(";")[2], data["TestName"]))

cols = ["Not alternating registers", "Alternating registers"]


fig, axes = plt.subplots(nrows=4, ncols=2)

pad = 5 # in points

for ax, col in zip(axes[0], cols):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')

for ax, row in zip(axes[:,0], test_types):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')

fig.set_figwidth(16)
fig.set_figheight(22)

for row_idx, test_type in enumerate(test_types):
    for col_idx, alternating in enumerate([False, True]):
        plot_to_axes(data, axes[row_idx, col_idx], test_type, alternating)

Plotting.savefig(arch_experiments, 'register-files.pdf', annotations_x_offset=0.075, annotations_y_offset=0.08, annotations_y_spacing=0.00625)